In [25]:
##### importing custom modules from the projects folder
import sys
from pathlib import Path
# Add project root to sys.path
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(PROJECT_ROOT))
import modules.scrapers as scrapers
import modules.helperModule as hf
##### --------------
from datetime import date

# ====================================
#                    PARAMS 
# ====================================
league = 'nfl'
export = True # export to csv files
today = date.today()
season = 2025
week = 0
inseason = False  # False for running proj, and rank scrapes in the offseason
#week = int(input("What week is it >>>>>> ? "))
print('Run date:',today, "\nWeek:", week, '\ninseason run:', inseason, '\nexport files:', export)
# ====================================

scraper = scrapers.scrapers(
    season = season,
    week = week,
    today = today
)

Run date: 2025-08-19 
Week: 0 
inseason run: False 
export files: True


# SCRAPE DATA
# =============

### INSEASON - Actual game scores and Game betting lines
##### run after the last game of the week before TNF game

In [ ]:
scraper.cbs_game_scores(inseason = inseason, export = export)
scraper.bp_lines(export = export)

### INSEASON or OFFSEASON - Projections and Rankings
##### offseason - run at any time in the offseason once sites put them up
##### inseason - run anytime Tues-Thurs before TNF to get the proj or ranks before cames start

In [ ]:
# in- and off-season
scraper.cbs_projections(inseason = inseason, export = export)
scraper.espn_projections(inseason = inseason, export = export)
scraper.nfl_projections(inseason = inseason, export = export)

In [28]:
# in- and off-season
scraper.ffp_ecr_rankings(inseason = inseason, export = export)
scraper.espn_rankings(inseason = inseason, export = export)
scraper.nfl_rankings(inseason = inseason, export = export)

c:\Users\jrbrz\Desktop\projects\projects\fantasyfootball\modules\scrapers.py:1542: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_df = pd.DataFrame(player_ranks, columns=hf.ranking_columns)


(384, 13)

###  OFFSEASON - ADPs

In [29]:
scraper.cbs_adp(export = export)
scraper.ffp_adp(export = export)
scraper.espn_adp(export = export)

(500, 10)

# PROCESS SCRAPED DATA

In [30]:
# hit db to standardize player, team, outlet, etc... ids 
scraper.generate_id_maps()

query successful
query successful
query successful
query successful
query successful
query successful


In [44]:
scraper.process_projections()

<string>:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<string>:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


5 missing players..
query successful


<string>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


query successful
successfully added data to player
inserts completed..
updates completed...


In [45]:
scraper.process_rankings()

<string>:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<string>:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


9 missing players..
query successful


<string>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


query successful
successfully added data to player
inserts completed..
updates completed...


In [46]:
scraper.process_adps()

cbs_adp_2025-00_2025-08-19.csv
espn_adp_2025-00_2025-08-19.csv
fp_adp_2025-00_2025-08-19.csv
raw
no missing players


<string>:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [ ]:
scraper.process_game_lines(bettingTableName = 'betting')

# load

In [48]:
print(
    'PROJ:', scraper.processed_dfs['projections'].shape,
    '\nRANK:', scraper.processed_dfs['rankings'].shape,
    '\nADP:', scraper.processed_dfs['adps'].shape)

PROJ: (1549, 56) 
RANK: (2931, 10) 
ADP: (2073, 6)


In [57]:
# remove nan player ids if the loads fail below 
table = 'rankings'

scraper.processed_dfs[table] = scraper.processed_dfs[table][
    scraper.processed_dfs[table]['playerId'].notna()
]
scraper.processed_dfs[table].shape

(2927, 10)

In [54]:
hf.export_database(
    dataframe=scraper.processed_dfs['projections'],
    database_table='projection', 
    connection_string=None, 
    if_exists='append'
)

successfully added data to projection


In [58]:
hf.export_database(
    dataframe=scraper.processed_dfs['rankings'],
    database_table='ranking', 
    connection_string=None, 
    if_exists='append'
)

successfully added data to ranking


In [59]:
hf.export_database(
    dataframe=scraper.processed_dfs['adps'],
    database_table='adp', 
    connection_string=None, 
    if_exists='append'
)

successfully added data to adp


In [ ]:

hf.export_database(
    dataframe=scraper.processed_dfs['lines'], 
    database_table='betting', 
    connection_string=None, 
    if_exists='append'
)

# scratch